In [1]:
%pwd
!pip install requests

### Filtering I (BM25 Retrieval)

In [2]:
import sys
import json
import requests
################################################################################
# Filtering
################################################################################
pretty = lambda x : json.dumps(x, indent=2, sort_keys=True)
solr_select = 'http://localhost:8983/solr/depcc-small/select?q='
train_path  = 'data/04-hyper/train.jsonl'
#train_path  = 'data/02-acl-arc/train.jsonl'
#train_path  = 'data/07-imdb/train.jsonl'
with open(train_path, 'r') as train_file:
    json_lines = []
    lines = train_file.readlines()
    for line in lines:
        j = json.loads(line)
        json_lines.append(j)
N = len(json_lines)
print(N)

# Query with training example
j = json_lines[0]
query = j['text'].replace(' ', '+')
#query = ""
#with open('data/02-acl-arc/lda_union.txt', 'r') as file:
#    query = file.read()
print(query)
#print(len(query.split()))
#sys.exit()
rp_retrieval = requests.get(solr_select + query).json()
cc_docs = (rp_retrieval['response']['docs'])
print('Number of retrieved documents: %d' % len(cc_docs))
cc_doc0 = json.loads(cc_docs[0]['_src_'])
cc_doc100 = ""
for i in range(10):
    cc_doc100 += json.loads(cc_docs[i]['_src_'])['text']
print(cc_doc100)
################################################################################

516
As+seen+on+The+Five+Police+Group+Boycotts+Ben+&amp;+Jerry's+After+Black+Lives+Matter+Endorsement+As+more+women+come+forward+with+allegations+of+sexual+misconduct+against+Donald+Trump,+the+Republican+nominee+is+lashing+out+at+the+mainstream+media,+which+he+claims+is+running+with+this+"total+fiction"+to+distract+from+Hillary+Clinton's+scandals.+At+a+rally+in+Greensboro,+N.C.,+Trump+said+these+accusations+are+part+of+a+conspiracy+involving+the+Clinton+campaign+and+what+he+called+“the+corrupt+media.”+Greg+Gutfeld+agreed+with+much+of+Trump's+assessment+on+"The+Five"+today,+saying,+"The+media+wants+Hillary+to+win.+There's+no+doubt+about+that."+Judge+Jeanine+on+Trump+Accusers:+'All+on+One+Day?+It's+a+Little+Too+Convenient'+Gutfeld+explained+that+Trump+has+actually+been+"set+up,"+as+he+was+given+countless+hours+of+coverage+and+"billions+of+dollars+of+free+media,"+only+to+have+the+rug+pulled+out+from+under+him.+"The+media+...+[and]+Hillary+and+Obama,+they+knew+elevating+Donald+Trump+-+givin

### Segmentation by 100 Tokens(Documents -> Passages)

In [3]:
################################################################################
# Segmentation
################################################################################
#val_url = cc_doc0['url']
#print(val_url)
#val_s3 = cc_doc0['s3']
#print(val_s3)
#cc_words = cc_doc0['text'].split()
'''


cc_words = cc_doc100.split()

cc_psgs = []
num_psgs = len(cc_words) // 100
print('Number of passages: %d ' % num_psgs)
for i in range(num_psgs):
    start = i * 100
    end   = -1 if (i == num_psgs-1) else (i+1) * 100
    psg = ' '.join(cc_words[start:end])
    dict = {'doc_id' : val_url, 'doc_text'  : psg,  'title': ''  }
    cc_psgs.append(dict)

print('Passage 0')
print(cc_psgs[0]['doc_text'])
'''

"\n\n\ncc_words = cc_doc100.split()\n\ncc_psgs = []\nnum_psgs = len(cc_words) // 100\nprint('Number of passages: %d ' % num_psgs)\nfor i in range(num_psgs):\n    start = i * 100\n    end   = -1 if (i == num_psgs-1) else (i+1) * 100\n    psg = ' '.join(cc_words[start:end])\n    dict = {'doc_id' : val_url, 'doc_text'  : psg,  'title': ''  }\n    cc_psgs.append(dict)\n\nprint('Passage 0')\nprint(cc_psgs[0]['doc_text'])\n"

### Segmentation by sentences(Documents -> Passages)

In [4]:
#!python -m spacy download en_core_web_sm
import spacy

nlp = spacy.load("en_core_web_sm", exclude=["parser"])
nlp.enable_pipe("senter")
doc = nlp(cc_doc100)
cc_psgs = []
psg = ''
num_tokens = 0
for sent in doc.sents:
    if num_tokens < 100:
        psg += sent.text
        num_tokens += len(sent)
    else:
#        print(num_tokens)
        cc_psgs.append({'doc_id' : '', 'doc_text'  : psg,  'title': ''  })
        num_tokens = 0
        psg = ''
#print(len(cc_psgs))

### Encoding Preparation

In [5]:
################################################################################
# Encoding Preparation
################################################################################

import csv
import subprocess

# Prepare cc_sample.tsv
f_out = 'emb/cc_sample.tsv'
with open(f_out, 'w') as output_file:
    dw = csv.DictWriter(output_file, cc_psgs[0].keys(), delimiter='\t')
    for psg in cc_psgs:
        dw.writerow(psg)
print('CC sample')
print(cc_psgs[0]['doc_text'])  
print()
# Prepare train_sample.tsv
num_train = len(json_lines)
train_psgs = []
for i in range(num_train):
    train_dict = {'doc_id': str(i), 'doc_text': json_lines[i]['text'], 'title': ''}
    train_psgs.append(train_dict)
    
print('train sample')
print(train_psgs[0]['doc_text'])

f_train_out = 'emb/train_sample.tsv'
with open(f_train_out, 'w') as output_file:
    dw = csv.DictWriter(output_file, train_psgs[0].keys(), delimiter='\t')
    for tp in train_psgs:
        dw.writerow(tp)

CC sample
STOKING FIRE : Anti - Choice Boycotts Continue , but Will Businesses Continue to Give In ?A little more than a year ago , the evangelical Christian group , Repent America , kicked off a campaign against Stericycle , a national medical waste disposal company .The goal ?To cripple abortion clinics by making it impossible for them to dispose of fetal remains , sharps , and other surgical throwaways .The organization calls Stericycle " a modern - day Auschwitz " and equates the company with German businesses that supported Hitler 's genocidal policies . "Government entities , police forces , military troops , and even civilians from a number of countries aided the Nazis in the destruction of the Jewish people , " their website begins .

train sample
As seen on The Five Police Group Boycotts Ben &amp; Jerry's After Black Lives Matter Endorsement As more women come forward with allegations of sexual misconduct against Donald Trump, the Republican nominee is lashing out at the mains

### Encode CC

In [6]:
# Encode cc_sample (tsv->embedding)
!bash emb/generate_embedding.sh cc_sample

Initialized host node122 as d.rank -1 on device=cuda, n_gpu=1, world size=1
16-bits training: False 
Reading saved model from /mnt/nfs/work1/696ds-s21/hhwang/DPR/data/checkpoint/retriever/multiset/bert-base-encoder.cp
model_state_dict keys odict_keys(['model_dict', 'optimizer_dict', 'scheduler_dict', 'offset', 'epoch', 'encoder_params'])
Overriding args parameter value from checkpoint state. Param = do_lower_case, value = True
Overriding args parameter value from checkpoint state. Param = pretrained_model_cfg, value = bert-base-uncased
Overriding args parameter value from checkpoint state. Param = encoder_model_type, value = hf_bert
Overriding args parameter value from checkpoint state. Param = sequence_length, value = 256
 **************** CONFIGURATION **************** 
batch_size                     -->   32
ctx_file                       -->   /mnt/nfs/work1/696ds-s21/hhwang/da/emb/cc_sample.tsv
device                         -->   cuda
distributed_world_size         -->   1
do_low

### Encode Train Sample

In [7]:
# Encode train_sample (tsv-> embedding)
!bash emb/generate_embedding.sh train_sample

Initialized host node122 as d.rank -1 on device=cuda, n_gpu=1, world size=1
16-bits training: False 
Reading saved model from /mnt/nfs/work1/696ds-s21/hhwang/DPR/data/checkpoint/retriever/multiset/bert-base-encoder.cp
model_state_dict keys odict_keys(['model_dict', 'optimizer_dict', 'scheduler_dict', 'offset', 'epoch', 'encoder_params'])
Overriding args parameter value from checkpoint state. Param = do_lower_case, value = True
Overriding args parameter value from checkpoint state. Param = pretrained_model_cfg, value = bert-base-uncased
Overriding args parameter value from checkpoint state. Param = encoder_model_type, value = hf_bert
Overriding args parameter value from checkpoint state. Param = sequence_length, value = 256
 **************** CONFIGURATION **************** 
batch_size                     -->   32
ctx_file                       -->   /mnt/nfs/work1/696ds-s21/hhwang/da/emb/train_sample.tsv
device                         -->   cuda
distributed_world_size         -->   1
do_

In [8]:
MAX_TR_PSGS = len(train_psgs)
MAX_CC_PSGS = len(cc_psgs)
print(MAX_TR_PSGS)
print(MAX_CC_PSGS)

516
98


### Nearest Neighbors

In [9]:
################################################################################
# Nearest Neighbor (FAISS)
################################################################################
#print(emb[0])
import numpy as np
import faiss

# Read CC embeddings (DATABASE)
cc_embeddings = np.load('emb/cc_sample_0.pkl', allow_pickle=True)
print(cc_embeddings[0][1].shape)  # Dimension of the embedding
nb = len(cc_embeddings) # database size
d = cc_embeddings[0][1].size
print(nb,d)
xb = np.zeros((nb,d), dtype='float32')
for i in range(nb):
    xb[i] = cc_embeddings[i][1]
print('Number of CC passages: %d' % nb)

# Read train embeddings (QUERY)
train_embeddings = np.load('emb/train_sample_0.pkl', allow_pickle=True)
print(train_embeddings[0][1].shape)  # Dimension of the embedding
nq = len(train_embeddings[:10]) # database size
d = train_embeddings[0][1].size
print(nq,d)
xq = np.zeros((nq,d), dtype='float32')
for i in range(nq):
    xq[i] = train_embeddings[i][1]

print('Number of train passages: %d' % nq)
print(xq)


index = faiss.IndexFlatL2(d)   # build the index

print('trained? %r' % index.is_trained)
index.add(xb)                  # add vectors to the index
print('Total number of indexed CC passages: ', index.ntotal)
print()
print('Using an indentical CC set')
k = 621                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print('================================================================')
print('4 nearest neighbors')
print(I)
print()
print('distances(sanity check)')
print(D)
print()


print('===============================================================')
print('Using the query(train set)')
D, I = index.search(xq, k)     # actual search
print('4 nearest neighbors')
print(I[:5])                   # neighbors of the 5 first queries
print('\ndistances')
print(D)
print()



(768,)
98 768
Number of CC passages: 98
(768,)
10 768
Number of train passages: 10
[[ 0.10329721 -0.17224678  0.32301617 ... -0.0395911   0.16416611
  -0.26728222]
 [ 0.29424196  0.25070143  0.22906397 ... -0.15961611 -0.08731496
  -0.24905857]
 [-0.15265112  0.08887525  0.2714078  ... -0.04350898  0.16684195
  -0.2133444 ]
 ...
 [-0.0047478   0.3250353   0.33790597 ... -0.00243538  0.22601245
  -0.557334  ]
 [ 0.18521078 -0.3216699   0.08558078 ...  0.07951526  0.18294588
   0.1011693 ]
 [ 0.35620537 -0.04019444  0.33199826 ...  0.0997697   0.35561934
  -0.31909522]]
trained? True
Total number of indexed CC passages:  98

Using an indentical CC set
4 nearest neighbors
[[ 0  1  2 ... -1 -1 -1]
 [ 1  0  6 ... -1 -1 -1]
 [ 2  0  6 ... -1 -1 -1]
 [ 3  4  6 ... -1 -1 -1]
 [ 4  6  3 ... -1 -1 -1]]

distances(sanity check)
[[0.0000000e+00 3.8639885e+01 5.1613029e+01 ... 3.4028235e+38
  3.4028235e+38 3.4028235e+38]
 [0.0000000e+00 3.8639885e+01 5.5799782e+01 ... 3.4028235e+38
  3.4028235e+38 

In [10]:
print('Train passage')
print(train_psgs[300]['doc_text'])
print()
print('CLOSEST passages in CC:')

for i in range(4):
    print('-------------------------------------------------------------')
    print('Closest %d' % i)
    closest = I[0][i]
    print(cc_psgs[closest]['doc_text'])

print('------------------------------------------------------------')
print('...')

for i in range(MAX_CC_PSGS-4, MAX_CC_PSGS):
    print('-------------------------------------------------------------')
    print('Farthest %d' % i)
    closest = I[0][i]
    print(cc_psgs[closest]['doc_text'])


Train passage
Surprisingly a lot of people are standing with Colin Kaepernick in his decision not to stand during the national anthem, "calling the Star Spangled Banner "weak" and saying that he doesn't truly love the song. Here is what he had to say on twitter: For those defending the current anthem, do you really truly love that song? I don't and I'm very good at singing it. Like, one of the best He continued to add: My vote is for America the Beautiful. Star spangled banner is a weak song anyway. And then you read this... https://t.co/iAE62FAbxj John Legend linked to an article titled, "Colin Kaepernick Is Righter Than You Know: The National Anthem Is A Celebration Of Slavery" The article makes the arguement that an unsung verse of the Francis Scott Key poem is a moral "atrocity." The author claims the end of the rarely sung third verse "literally celebrated the murder of African-Americans." The verse reads: No refuge could save the hireling and slave From the terror of flight or th